In [1]:
/home/orchids/ms_cdsw_final_data/PartB_Telugu/Train              -- server file path

SyntaxError: invalid syntax (<ipython-input-1-7fdd41fe4247>, line 1)

In [3]:
import keras
import tensorflow as tf


config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.compat.v1.Session(config=config) 
keras.backend.set_session(sess)

In [1]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0']


In [2]:
import tensorflow as tf
tf.test.is_gpu_available()
# tf.test.is_built_with_cuda()

False

In [16]:
import python_speech_features
import numpy as np
import soundfile as sf
import pickle
import cv2
import glob

1. Take the 1000 files and compute 200ms (3200) samples for all and save it in .pkl  #chcek all 200ms have 3200 samples
2. Compute mfcc for all 200 ms frames #check all have 19 frames
3. for i in range(0,len(tel_feat)):
  tel_feat[i] = numpy.stack( tel_feat[i], axis=0 )

4.t = np.array(tel_feat[0:2467])
t.shape _: samples,19,39

samples: Number of 200 ms segments

time steps: No of frames in it

features :  mfcc

Step 1: Take 1000 files and find 200ms telugu segments in each

In [17]:
#Make Dataset ready
import csv
import pandas as pd
list = ['File Name','Sequence']
df = pd.read_csv("/home/orchids/Documents/ms_sld_env/PartB_Telugu/Train/Transcription_LT_Sequence_Frame_Level_200_actual.tsv", delimiter="\t",names=list)
df['File Name'] = df['File Name'].apply(lambda x: str(x))
for x in range(0,len(df['File Name'])):
  df['File Name'][x] = str(str(df['File Name'][x]).zfill(9))
df

,File Name,Sequence
0,000010010,SSTTTTTTTTTTEEEEEETTSEEEEESSSSTTTTTTSSSSSSSSTT...
1,000010012,SSEEEETTSTTTTTTTTTTTTTTTTTTTTTSS
2,000010017,SSEEETTTTTTTTTTTTTTTTTTTTTTS
3,000010023,SSTTTTTTTTTTTTTTTTTTEESSSSSSEEEEEEETTTTTTTTSS
4,000010030,SSTTTTTTTTTTEETTTS
...,...,...
8220,102280462,SSTTTTTTEEEEETTTTTTTSS
8221,102280464,SSTTTTTTEETEETTSS
8222,102280484,SSTTTTTEETTETTTTT
8223,102280489,SSEEETTTTTTTTEETTTSS


Take all Telugu segments

In [ ]:
path = "/content/gdrive/MyDrive/MS_CD_SW/PartB_Telugu/Train/Audio/*.*"

In [4]:
#Find all telugu 200 ms frames
tel_frames= []
cnt = 0

fin = []
#Run for file name in df
for file_name in df['File Name']:
  cnt = cnt+1
  #Combine name with path
  file_path ="/home/orchids/Documents/ms_sld_env/PartB_Telugu/Train/Audio/"+file_name+'.wav'
  #Read the file
  data, fs = sf.read(file_path)
 
  #------Get sequence of Telugu---------#
  seq = df.loc[df['File Name'] == file_name, 'Sequence'].iloc[0]

    #---------Find indexes of Telugu segments in seq-----------------#
  index = []
  for i in range(len(seq)):
      if (seq[i] == 'E'):
          index.append(i)
  #find the speech data from each index and take 200 ms data, i.e 0.2*16000 = 3200
  frame = int(0.2*fs)
  #fin = []
  for i in index:
    if len(data[frame*i:frame*i+frame])==3200:
      fin.append((data[frame*i:frame*i+frame]).tolist())
  #APpend in final list


In [5]:
len(fin)

33285

In [6]:
for x in fin:

  if len(x)!=3200:
  
    print("Founf")
    print(len(x))
  

MFCC vectors

In [7]:
final_feat_vec = []
for data in fin:
  data = np.array((data))
  feat_file = []
  mfcc_feat = python_speech_features.mfcc(data,16000,winlen=0.02,winstep=0.01)
  #-------Compute delta coef-----#
  N=2
  delta = python_speech_features.delta(mfcc_feat, 2)
  #--------Compute delta-delta coeff-------#
  delta_delta = python_speech_features.delta(delta, 2)

  for frame in range(0,len(mfcc_feat)):
    #frame opened
    #concatenate for each frame : 13 mfcc + 13 delta + 13 delta-delta = 39 features
    feat_ = np.concatenate((mfcc_feat[frame],delta[frame],delta_delta[frame]))
    feat_file.append(feat_)

  final_feat_vec.append(feat_file)

In [8]:
for x in final_feat_vec:

  if len(x)!=19:
  
    print("Founf")
    print(len(x))
  

In [9]:
len(final_feat_vec[0][1])

39

In [10]:
feat_file = open("/home/orchids/Documents/ms_sld_env/PartB_Telugu/Train/english_feat.pkl", "wb")
pickle.dump(final_feat_vec, feat_file)
feat_file.close()

In [11]:
import pickle
feat_file = open("/home/orchids/Documents/ms_sld_env/PartB_Telugu/Train/english_feat.pkl", "rb")
tel_feat = pickle.load(feat_file)


In [12]:
len(tel_feat[0])

19

Step 3

In [ ]:
import numpy as np

In [17]:
for i in range(0,len(tel_feat)):
  tel_feat[i] = np.stack( tel_feat[i], axis=0 )

t = np.array(tel_feat)

t.shape

(195868, 19, 39)

t = np.array(tel_feat[0:2467]) t.shape _: samples,19,39 samples: Number of 200 ms segments time steps: No of frames in it features : mfcc

In [ ]:
#t.shape _: samples,19,39 
#samples: Number of 200 ms segments 
#time steps: No of frames in it 
#features : mfcc

In [13]:
t = np.array(tel_feat)

In [14]:
t.shape

(195868, 19, 39)

In [ ]:
len(t)

24678

In [ ]:
# lstm autoencoder recreate sequence
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
#from keras.utils import plot_model

In [ ]:
sequence = t

In [ ]:

# define input sequence
#sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
#n_in = len(sequence)
#sequence = sequence.reshape((1, n_in, 1))
# define model
n_in = 19

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,39)))
model.add(RepeatVector(n_in))
#k fu

model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(39)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(sequence, sequence, epochs=10, verbose=0)
#plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
# demonstrate recreation
#yhat = model.predict(sequence, verbose=0)
#print(yhat[0,:,0])

In [ ]:
inp = model.input


In [ ]:
t[0].shape

(19, 39)

In [ ]:
inp

<KerasTensor: shape=(None, None, 19, 39) dtype=float32 (created by layer 'input_3')>

In [ ]:
import tensorflow as tf


In [ ]:
out = model.layers[0].output


In [ ]:
tf.print(out,output_stream=sys.stderr)

NameError: ignored

In [ ]:
input1

<KerasTensor: shape=(None, 19, 39) dtype=float32 (created by layer 'lstm_2_input')>

In [ ]:
inp = tf.keras.Input(shape = ( 19, 39))
inp

<KerasTensor: shape=(None, 19, 39) dtype=float32 (created by layer 'input_5')>

In [ ]:
input1 = model.input               # input placeholder

output1 = [layer.output for layer in model.layers]# all layer outputs

fun = K.function([input1, K.learning_phase()],output1)# evaluation function

# Testing

#t = np.random.random(input_shape)[np.newaxis,...]

layer_outputs = fun([t, 1.])

print(layer_outputs)

ValueError: ignored

In [ ]:
from keras import backend as K

#inp = tf.keras.Input(model.input)
#shape=None
inp = tf.keras.Input(shape = (19, 39))

outputs = [model.layers[0].output]

functor = K.function([inp]+ [K.learning_phase()], outputs )

layer_outs = functor([t, 0.])


ValueError: ignored

In [ ]:
plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')

In [ ]:
yhat = model.predict(sequence, verbose=0)

In [ ]:
len(yhat)

24678

In [ ]:
len(yhat[0])

19

In [ ]:
len(yhat[0][1])

39

In [ ]:
sequence[0][1]

array([-8.36641164e+00, -1.06482045e+01, -2.81928372e+01, -1.83197708e+00,
        3.96222225e+00,  1.42878538e+01,  1.32618103e+01,  1.63838714e+01,
        1.07488195e+01,  6.12976276e+00,  1.26734297e+00,  5.12138307e+00,
        4.78382373e+00, -1.34855932e-01, -7.80031789e-01,  4.65989977e-01,
        4.46870403e-01, -1.88519525e+00, -1.10102742e-01, -8.79461477e-01,
        6.89907526e-02,  1.56513016e+00, -2.73388275e-01,  1.18724548e+00,
       -8.20394642e-01, -2.56406880e+00,  1.37729192e-02,  4.08821029e-01,
       -7.02858569e-02,  3.83083283e-01,  6.44138632e-01, -2.19186589e-01,
        3.24915361e-01, -7.55712583e-02, -1.47974282e+00, -5.92094984e-02,
        4.75678628e-01,  4.34985124e-03, -5.61092737e-01])

In [ ]:
yhat[0][1]

array([-7.5368161e+00, -1.6584654e+01, -2.6660812e+01, -3.5798349e+00,
        3.3973074e+00,  8.8104019e+00,  1.1914141e+01,  9.7233820e+00,
        8.5710449e+00,  3.5628817e+00, -2.1006153e+00, -2.0576826e-01,
        8.6096239e-01,  4.7611721e-02,  7.5080574e-01, -2.9001087e-01,
       -2.3796082e-02,  5.0927854e-01, -1.7457530e-01,  1.1901805e+00,
       -8.8306510e-01, -1.4283777e+00, -4.9093804e-01,  7.5566930e-01,
        5.4270291e-01, -8.9968383e-01,  6.1991073e-02,  8.5394764e-01,
       -4.0522214e-02, -3.5489821e-01, -6.6299003e-01, -7.5054541e-02,
       -5.1046735e-01, -4.4891870e-01, -4.5307732e-01, -7.4565746e-02,
       -4.7260020e-02, -4.7343540e-01,  5.2391198e-03], dtype=float32)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               56000     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 19, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 19, 100)           80400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 19, 39)            3939      
Total params: 140,339
Trainable params: 140,339
Non-trainable params: 0
_________________________________________________________________


Autoencoder

In [ ]:
import pickle
feat_file = open("/content/gdrive/MyDrive/MS_CD_SW/PartB_Telugu/Train/telugu_feat_1000.pkl", "rb")
tel_feat = pickle.load(feat_file)

In [ ]:
import numpy as np

for i in range(0,len(tel_feat)):
  tel_feat[i] = np.stack( tel_feat[i], axis=0 )

sequence = np.array(tel_feat)

sequence.shape

(24678, 19, 39)

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
n_in = 19

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,39)))
model.add(RepeatVector(n_in))

model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(39)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(sequence, sequence, epochs=10, verbose=1)

Epoch 1/10
772/772 [==============================] - 27s 30ms/step - loss: 114.8980
Epoch 2/10
772/772 [==============================] - 23s 30ms/step - loss: 62.4502
Epoch 3/10
772/772 [==============================] - 23s 30ms/step - loss: 52.7933
Epoch 4/10
772/772 [==============================] - 23s 30ms/step - loss: 46.1371
Epoch 5/10
772/772 [==============================] - 23s 30ms/step - loss: 41.6316
Epoch 6/10
772/772 [==============================] - 23s 30ms/step - loss: 38.9823
Epoch 7/10
772/772 [==============================] - 23s 30ms/step - loss: 37.0952
Epoch 8/10
772/772 [==============================] - 23s 30ms/step - loss: 35.6183
Epoch 9/10
772/772 [==============================] - 23s 30ms/step - loss: 34.4521
Epoch 10/10
772/772 [==============================] - 23s 30ms/step - loss: 33.1330


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               56000     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 19, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 19, 100)           80400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 19, 39)            3939      
Total params: 140,339
Trainable params: 140,339
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs

#get output of each layer
functors = [K.function([inp], [out]) for out in outputs]    # evaluation functions

# Testing
#test = np.random.random(input_shape)[np.newaxis,...]
test = sequence
layer_outs = [func([test]) for func in functors]
print (layer_outs)


In [ ]:
print(len(layer_outs))  #4 layers

4


In [ ]:
print(len(sequence))

24678


In [ ]:
print(len(layer_outs[0]))

1


In [ ]:
a = layer_outs[2]

In [ ]:
len(a[0])

24678

In [ ]:
layer_outs

NameError: ignored

In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components = 2)
gmm.fit(layer_outs)


ValueError: ignored

In [ ]:
#log-likelihood score
print(gmm.lower_bound_)
